In [44]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
#Load the csv file
credit_card_data = pd.read_csv('/content/drive/MyDrive/Data Science/Projects/10 Credit Card Fraud Detection/creditcard.csv')
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
credit_card_data.shape

(284807, 31)

In [10]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [11]:
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [14]:
#distribution of legit transaction and fraudulant transaction
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


This dataset is highly imbalanced,
0 - normal , 1 - fraud

In [18]:
#seperate the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [21]:
#statistical measures of the data
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [22]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [28]:
#compare the values for both transactions
credit_card_data.groupby('Class').mean()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under-Sampling

1.   Build a sample data set  containing similar distribution of normal Transaction and fraud transactions
2. Number of Fraud Transactions - 492


In [29]:
legit_sample = legit.sample(n=492) #random sampling

In [30]:
legit_sample.shape

(492, 31)

Concatenating 2 data frames

In [31]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)
new_dataset.shape

(984, 31)

In [32]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
105175,69394.0,1.064630,-0.272178,0.744352,1.349040,-0.040236,1.614714,-0.567454,0.450953,0.944825,-0.283213,-0.311956,1.722339,0.674416,-0.771856,-2.405754,-0.645053,0.054145,-0.572917,0.888932,-0.050538,-0.413865,-0.768865,-0.107540,-1.130239,0.568315,-0.459958,0.086035,0.013312,39.00,0
108834,71097.0,-1.621272,0.145210,1.543398,2.168138,-0.107901,0.917462,-0.659340,0.671751,0.552853,-0.252034,-1.612935,0.827955,0.404245,-0.622494,-0.968996,-1.636061,1.323167,-0.847577,2.181811,-0.140127,-0.279541,-0.431736,0.069718,-0.393842,-0.342638,-0.333941,-0.315446,0.269992,12.49,0
136487,81730.0,-2.815238,-0.255333,1.252226,0.613973,0.529399,-0.795113,0.246524,0.398968,-0.225499,-0.431691,0.393337,0.153583,-1.690182,0.509763,-1.331534,0.208122,-0.316223,0.243746,-0.012997,-1.190274,-0.254830,0.000040,0.639906,0.299184,0.945501,-0.399915,-0.085280,-0.248610,19.26,0
125421,77668.0,0.465919,-2.993640,-1.272432,-1.229200,-1.113336,0.456084,-0.107029,-0.149254,-1.866562,1.315565,-0.734614,-1.188278,-0.402370,0.078060,-0.771508,-0.153514,0.074824,0.986342,0.811339,0.852048,0.074783,-0.687421,-0.819714,-1.375586,0.626155,-0.034101,-0.108433,0.082369,605.48,0
178622,123710.0,-0.003931,0.750408,0.179211,-0.763513,0.643163,-0.652448,0.854284,-0.023420,-0.244875,-0.230193,0.448140,0.607274,-0.237625,0.335308,-1.054715,0.205441,-0.762053,-0.121016,0.328191,-0.022875,-0.229590,-0.538261,0.025773,-0.348949,-0.482794,0.141436,0.240384,0.084289,1.99,0


In [33]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,2.115795,-5.417424,-1.235123,-6.665177,0.401701,-2.897825,-4.570529,-1.315147,0.391167,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,2.858466,-3.096915,-0.792532,-5.210141,-0.613803,-2.155297,-3.267116,-0.688505,0.737657,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,1.794969,-2.775022,-0.418950,-4.057162,-0.712616,-1.603015,-5.035326,-0.507000,0.266272,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,1.933520,-5.030465,-1.127455,-6.416628,0.141237,-2.549498,-4.614717,-1.478138,-0.035480,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,0.491140,0.728903,0.380428,-1.948883,-0.832498,0.519436,0.903562,1.197315,0.593509,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [35]:
new_dataset['Class'].value_counts()

,count
Class,
0,492
1,492


In [36]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,93921.817073,-0.046089,-0.016856,0.007686,-0.014855,0.063408,0.048515,-0.001157,-0.090271,0.011165,0.032968,-0.028233,-0.026302,-0.030993,0.031079,0.029524,-0.010793,-0.004973,0.041280,-0.003827,-0.012548,0.016287,0.025128,0.009922,-0.01758,0.003181,0.015771,0.001164,0.011178,94.676484
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.10513,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the data into Features and Targets

In [39]:
x = new_dataset.drop(['Class'], axis=1)
y = new_dataset['Class']

Split the data into training and test

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, stratify=y,random_state=2)
x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape

((984, 30), (787, 30), (197, 30), (787,), (197,))

Model Training, Logistric Regression

In [46]:
model = LogisticRegression()
#training the Logistic Regression
model.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation,
Accuracy score

In [48]:
x_train_predicition = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_predicition, y_train)
print('Accuracy on training data: ', training_data_accuracy*100)

Accuracy on training data:  93.90088945362135


In [49]:
x_test_predicition = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_predicition, y_test)
print('Accuracy on test data: ', test_data_accuracy*100)

Accuracy on test data:  91.87817258883248


Building a predictive system

In [60]:
input_data = x_test.iloc[1].values.reshape(1,-1)
prediction = model.predict(input_data)
print(prediction)

if prediction == 0:
  print('The transaction is legit')
else:
  print('The transaction is fraud')

[0]
The transaction is legit


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [62]:
x_test.iloc[1]

,240102
Time,150427.000000
V1,2.376404
V2,-1.122618
V3,-1.557956
V4,-1.726235
V5,-0.651332
V6,-0.883377
V7,-0.671633
V8,-0.441828
V9,-1.701652
